In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
df = pd.read_csv('population_by_zip_2010.csv')

In [17]:
zip_pop = df.groupby('zipcode').sum()[['population']]

In [18]:
zip_pop = zip_pop.reset_index()

In [21]:
sta = pd.read_csv('station_locations.csv')

In [24]:
sta['zipcode'] = sta['station'].apply(lambda x: int(x.split(', ')[-1]))

In [25]:
sta.merge(zip_pop, on='zipcode')

,latatude,longitude,raw,station,zipcode,population
0,37.797385,-122.265300,"{u'lat': 37.797385, u'lng': -122.2653001}","800 Madison Street, Oakland, CA, 94607",94607,74934
1,37.804875,-122.295314,"{u'lat': 37.8048749, u'lng': -122.2953143}","1451 7th Street, Oakland, CA, 94607",94607,74934
2,NaN,NaN,"1750+Sacramento+Street,+Berkeley,+CA,+94702","1750 Sacramento Street, Berkeley, CA, 94702",94702,47937
3,37.775150,-122.224263,"{u'lat': 37.7751503, u'lng': -122.224263}","3401 East 12th Street, Oakland, CA, 94601",94601,150882
4,37.615470,-122.389570,"{u'lat': 37.61546999999999, u'lng': -122.38957}","International Terminal, Level 3, San Francisco...",94128,207
5,37.870538,-122.268136,"{u'lat': 37.8705381, u'lng': -122.2681359}","2160 Shattuck Avenue, Berkeley, CA, 94704",94704,76776
6,37.557385,-121.976762,"{u'lat': 37.5573846, u'lng': -121.976762}","2000 BART Way, Fremont, CA, 94536",94536,206370
7,37.701644,-121.899195,"{u'lat': 37.7016441, u'lng': -121.8991946}","5801 Owens Dr., Pleasanton, CA, 94588",94588,92526
8,37.844850,-122.250987,"{u'lat': 37.8448503, u'lng': -122.2509873}","5660 College Avenue, Oakland, CA, 94618",94618,48138
9,38.017481,-121.888703,"{u'lat': 38.0174808, u'lng': -121.8887032}","2099 Railroad Avenue, Pittsburg, CA, 94565",94565,253923


In [28]:
blocks = pd.read_csv('/Users/ryand/Downloads/All_Geographies/e20171ma0117000.txt')

In [30]:
import os

In [35]:
blockpop_filelist = []
blockhome_filelist = []
for a,b,c in os.walk('/Users/ryand/Downloads/data/tab4/sumfile/prod/2012thru2016/group2'):
    for file in c:
        if file[0] == 'e':
            if file[8:12]=='0001':
                blockpop_filelist.append(file)
            elif file[8:12]=='0002':
                blockhome_filelist.append(file)


In [39]:
df_pop = pd.DataFrame()
for file in blockpop_filelist:
    try:
        df = pd.read_csv('/Users/ryand/Downloads/data/tab4/sumfile/prod/2012thru2016/group2/' + file)
        df_pop = df_pop.append(df)
    except:
        print(file)

e20165us0001000.txt


In [38]:
df_pop

,ACSSF,2016e5,wi,000,0001,0004158,453,339
0,ACSSF,201600000.0,wi,0,1,4159,360.0,294.0
1,ACSSF,201600000.0,wi,0,1,4160,800.0,673.0
2,ACSSF,201600000.0,wi,0,1,4161,999.0,575.0
3,ACSSF,201600000.0,wi,0,1,4162,354.0,344.0
4,ACSSF,201600000.0,wi,0,1,4163,793.0,458.0
5,ACSSF,201600000.0,wi,0,1,4164,901.0,595.0
6,ACSSF,201600000.0,wi,0,1,4165,645.0,308.0
7,ACSSF,201600000.0,wi,0,1,4166,301.0,108.0
8,ACSSF,201600000.0,wi,0,1,4167,229.0,107.0
9,ACSSF,201600000.0,wi,0,1,4168,919.0,396.0
